In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/kaggle/input/translation/translation_train.csv') # ,index_col='Language'
df.head(10)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23406 entries, 0 to 23405
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   English  23406 non-null  object
 1   Arabic   23406 non-null  object
dtypes: object(2)
memory usage: 365.8+ KB


In [3]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [34]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

In [35]:
import unicodedata
import regex  # Better support for Unicode regular expressions

def normalizeArabic(text):

    text = unicodeToAscii(text.lower().strip())
    # Normalize Arabic characters
    text = regex.sub(r'[\p{Mn}\p{Sk}]+', '', unicodedata.normalize('NFKD', text))

    # Remove non-letter, non-space characters
    text = regex.sub(r'[^\p{L}\s]', '', text)

    # Normalize whitespace
    text = regex.sub(r'\s+', ' ', text)

    return text.strip()

# Example usage
arabic_text = "مرحباً بكم في العالم العربي! %&&"
normalized_text = normalizeArabic(arabic_text)
print(normalized_text)  # Output: "مرحبا بكم في العالم العربي"

مرحبا بكم في العالم العربي


In [36]:
import csv

def readLangs(lang1, lang2, reverse=True):
    print("Reading lines...")

    # Open the CSV file
    with open('/kaggle/input/translation/translation_train.csv', newline='', encoding='utf-8') as csvfile:
        # Create a CSV reader object
        reader = csv.reader(csvfile)
        
        # Read the rows of the CSV file
        lines = [row for row in reader]

    # Split every line into pairs and normalize
    pairs = [[normalizeArabic(s) for s in l] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        print(input_lang)
        output_lang = Lang(lang1)
        print(output_lang)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


In [47]:
# some filtering based on sentence length and sentence start prefix
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re ","This","That"
)

ara_prefixes = ( 'هذه','انا', 'لا')

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes) # turning off filtering 


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [75]:
def prepareData(lang1, lang2, reverse=True):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(f'input lang is :{input_lang.name}')
    print(input_lang.name, input_lang.n_words)
    print(f'output lang is :{output_lang.name}')
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('english', 'arabic', True)
print(random.choice(pairs))

Reading lines...
Read 23407 sentence pairs
Trimmed to 316 sentence pairs
Counting words...
Counted words:
input lang is :arabic
arabic 563
output lang is :english
english 402
['انه وسيم و ذكي', 'he is handsome and clever']


In [49]:
# lets take an example
print(random.choice(pairs))

['سالعب كرة المضرب غدا', 'i am going to play tennis tomorrow']


In [50]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [51]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

In [52]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights


In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None, teacher_forcing_ratio=0.5):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if use_teacher_forcing and target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1)
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach() # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions

    def forward_step(self, input, hidden, encoder_outputs):
        embedded = self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights


In [62]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('eng', 'ara', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

In [63]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [64]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [65]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [66]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [67]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

In [68]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [71]:
hidden_size = 256
batch_size = 16

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, encoder,decoder, n_epochs = 500, print_every=5, plot_every=5)

Reading lines...
Read 23407 sentence pairs
Trimmed to 316 sentence pairs
Counting words...
Counted words:
ara 563
eng 402
0m 1s (- 3m 0s) (5 1%) 2.5231
0m 3s (- 3m 0s) (10 2%) 1.6206
0m 5s (- 2m 59s) (15 3%) 1.0788
0m 7s (- 2m 56s) (20 4%) 0.6336
0m 9s (- 2m 59s) (25 5%) 0.2595
0m 11s (- 2m 56s) (30 6%) 0.1090
0m 13s (- 2m 53s) (35 7%) 0.0561
0m 14s (- 2m 51s) (40 8%) 0.0350
0m 16s (- 2m 49s) (45 9%) 0.0256
0m 18s (- 2m 47s) (50 10%) 0.0263
0m 20s (- 2m 44s) (55 11%) 0.0174
0m 22s (- 2m 43s) (60 12%) 0.0137
0m 24s (- 2m 41s) (65 13%) 0.0144
0m 25s (- 2m 39s) (70 14%) 0.0119
0m 27s (- 2m 37s) (75 15%) 0.0113
0m 29s (- 2m 35s) (80 16%) 0.0103
0m 31s (- 2m 33s) (85 17%) 0.0093
0m 33s (- 2m 31s) (90 18%) 0.0091
0m 35s (- 2m 29s) (95 19%) 0.0088
0m 36s (- 2m 27s) (100 20%) 0.0082
0m 38s (- 2m 25s) (105 21%) 0.0087
0m 40s (- 2m 23s) (110 22%) 0.0078
0m 42s (- 2m 22s) (115 23%) 0.0069
0m 44s (- 2m 20s) (120 24%) 0.0082
0m 46s (- 2m 18s) (125 25%) 0.0077
0m 47s (- 2m 16s) (130 26%) 0.0073
0m 4

In [72]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

> هن اخواتي
= they are my sisters
< they are my sisters <EOS>

> حظه يسبق ذكاءه
= he is more lucky than clever
< he is more lucky than clever <EOS>

> انك لست طالبا
= you are not a student
< you are not a student <EOS>

> تقول انك تتعمد اخفاء مظهرك الحسن
= you are saying you intentionally hide your good looks
< you are saying you intentionally hide your good looks <EOS>

> انه قلق بسبب مرض والده
= he is concerned about his fathers illness
< he is concerned about his fathers illness <EOS>

> انت امي
= you are my mother
< you are my mother <EOS>

> تحبني كل عايلتي
= i am loved by all my family
< i am loved by all my family <EOS>

> انا من الاكوادور
= i am from ecuador
< i am from ecuador <EOS>

> هو رجل حكمة
= he is a man of wit
< he is a man of wit <EOS>

> انت لست مسوولا عن تلك الفوضى
= you arent responsible for that mess
< you arent responsible for that mess <EOS>



In [24]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

> i dont think tom would want to do that
= لا اظنن توم يريد فعل ذلك
< لا اظنن توم يريد فعل ذلك <EOS>

> i read his book
= انا اقرا كتابه
< انا اقرا كتابه على الاطلاق <EOS>

> im sure that she will come back soon
= انا متاكد من انها ستعود قريبا
< انا متاكد من انها ستعود قريبا <EOS>

> im serious
= انا لا امزح
< انا لا احاول ان <EOS>

> im really hungry
= انا جايع جدا
< انا جايع جدا في الصباح <EOS>

> i cant see anything
= لا ارى شييا
< لا استطيع ابتكار ارى <EOS>

> im trying to sleep
= انا احاول ان انام
< انا احاول ان انام احاول <EOS>

> no one shall be arbitrarily deprived of his property
= لا يجوز تجريد احد من ملكه تعسفا
< لا يجوز تجريد احد من ملكه تعسفا <EOS>

> i like animals for example cats and dogs
= انا احب الحيوانات على سبيل المثال القطط والكلاب
< انا احب الحيوانات على سبيل المثال القطط والكلاب <EOS>

> i dont know when hell be here
= لا اعرف متى سيكون هنا
< لا اعرف متى سيكون هنا <EOS>



In [73]:
# Save encoder
torch.save(encoder.state_dict(), './ara_encoder.pth')

# Save decoder
torch.save(decoder.state_dict(), './ara_decoder.pth')


In [ ]:
encoder.load_state_dict(torch.load('./eng_to_ara_encoder.pth'))
decoder.load_state_dict(torch.load('./eng_to_ara_decoder.pth'))

def evaluateOneSentence(encoder, decoder, sentence, input_lang, output_lang):
    print('>', sentence)
    output_words, _ = evaluate(encoder, decoder, sentence, input_lang, output_lang)
    output_sentence = ' '.join(output_words)
    print('<', output_sentence)

# Example usage:
input_lang, output_lang, pairs = prepareData('english', 'arabic', False)
encoder = encoder # initialize your encoder
decoder = decoder # initialize your decoder
# input_sentence = "انا لا اشعر بالعطش"
input_sentence = "know what"
evaluateOneSentence(encoder, decoder, input_sentence, input_lang, output_lang)


In [ ]:
import random
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

def evaluateRandomly(encoder, decoder, n=10):
    references = []  # List to store reference translations
    candidates = []  # List to store candidate translations
    
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')
        
        references.append(pair[1].split())  # Add reference translation to references list
        candidates.append(output_words[:-1])  # Remove <EOS> token from candidate translation and add to candidates list
    
    # Calculate BLEU score with smoothing function
    smoothing_function = SmoothingFunction().method4
    bleu_score = nltk.translate.bleu_score.corpus_bleu(references, candidates)
    print("BLEU Score:", bleu_score)

encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)


In [29]:
import random

def evaluateSpecificSentence(encoder, decoder, sentence, input_lang, output_lang):
    print('>', sentence)
    output_words, _ = evaluate(encoder, decoder, sentence, input_lang, output_lang)
    output_sentence = ' '.join(output_words)
    print('<', output_sentence)
    print('')


In [32]:
encoder.eval()
decoder.eval()
evaluateSpecificSentence(encoder, decoder,"no please",input_lang,output_lang)

> no please
< لا يوجد احد من ملكه <EOS>

